In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import os
from itertools import product
import math
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time

sys.path.append(os.path.abspath(".."))

def symmetrize(A):
    n = A.shape[0]
    B = A.copy()
    B[np.tril_indices(n)] = B.T[np.tril_indices(n)]
    return B

all_classes = {}

all_classes["resnet"] = ['regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf', 'regnet_x_400mf',
            'regnet_x_800mf', 'regnet_x_8gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf',
            'regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf', 'resnet18',
            'resnext50_32x4d', 'wide_resnet50_2'] # convolutional residual networks (skip connections) # 17

all_classes["efficientnet"] = ['efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4',
            'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7'] # convolutional # 8

all_classes["alexnet"] = ["alexnet"] # convolutional # 1

all_classes["googlenet"] = ["googlenet"] # convolutional # 1

all_classes["inception"] = ["inception"] # convolutional # 1

all_classes["mnasnet"] = ["mnasnet"] # convolutional # 1

all_classes["vgg16"] = ["vgg16"] # convolutional # 1

all_classes["mobilenet"] = ['mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small'] # convolutional # 3

all_classes["convnext"] = ['convnext_small', 'convnext_tiny'] # depthwise convolutional that were designed to compete with transformers # 2

# Total 35 models

classes = {}
for c in all_classes:
    if len(all_classes[c]) > 1:
        classes[c] = all_classes[c]
num_classes = len(classes)
class_names = np.array(list(classes.keys()))

models_pretrained_folder = "/content/drive/MyDrive/UKP/imagenet_experiments/models/"

pretrained_model_names = ['regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf', 'regnet_x_400mf',
            'regnet_x_800mf', 'regnet_x_8gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf',
            'regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf', 'resnet18',
            'resnext50_32x4d', 'wide_resnet50_2','efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4',
            'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7','alexnet','googlenet','inception','mnasnet','vgg16','mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small',
               'convnext_small', 'convnext_tiny']


num_pretrained_models = len(pretrained_model_names)

# loading test distance file and seeing its entries
datafilepath = "/content/drive/MyDrive/UKP/imagenet_experiments/distances_torch/val/pretrained/alexnet_pretrained_rep_convnext_small_pretrained_rep.npz"
data = np.load(f"{datafilepath}")
print(data.files)

distnames = data.files

####################################################################################################################3

# 0.1. Show distance matrices for different distance functions
subset = "val"
distances_pretrained_folder = "/content/drive/MyDrive/UKP-experiments/imagenet_experiments/distances_torch/val/pretrained"
stats = np.load(f"{distances_pretrained_folder}/stats.npz")

# Code to extract each different distance into its own array
dist_array = {distname: [] for distname in distnames}
distfilenames = np.sort(os.listdir(distances_pretrained_folder))

total_distfilenames = len(distfilenames)
print(f"Total number of distance files: {total_distfilenames}")


In [ ]:
for i in range(len(pretrained_model_names)):
  first_model_name = pretrained_model_names[i]
  for j in range(i+1,len(pretrained_model_names)):
    second_model_name = pretrained_model_names[j]
    check_string1 = f"{first_model_name}_pretrained_rep_{second_model_name}_pretrained_rep.npz"
    check_string2 = f"{second_model_name}_pretrained_rep_{first_model_name}_pretrained_rep.npz"
    print(f"Checking File ({i} {first_model_name},{j} {second_model_name})")

    for filename in distfilenames:
      if filename == check_string1 or filename == check_string2:
        file_path = os.path.join(distances_pretrained_folder, filename)
        data = np.load(file_path)

        for distname in distnames:
            if distname in data:
                dist_array[distname].append(data[distname])


In [ ]:
for distname in distnames:
  if "GULP" not in distname and "UKP" not in distname:
    dist_array[distname] = np.array(dist_array[distname])
  else:
    dist_array[distname] = np.sqrt(np.array(dist_array[distname]))
    print(f"distname = {distname}, length = {len(dist_array[distname])}")

In [ ]:
np.savez(f"/content/drive/MyDrive/UKP-experiments/imagenet_experiments/distances_torch/val/pretrained_categorized/all_distances_categorized_correctedfinal.npz", **dist_array)